<a href="https://colab.research.google.com/github/DEVjspf/ceu_facturas_tfa/blob/main/dev_facturas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyecto: Extracción de datos de facturas eléctricas
## Objetivo:
Extraer y gestionar información clave de facturas eléctricas para optimizar la asesoría y gestión mensual.

In [ ]:
# Instalación de librerías necesarias
#!pip install langchain langchain-community langchain-experimental ollama PyMuPDF pydantic sqlalchemy psycopg[binary] faiss-cpu
#!pip install chromadb


### 📘 1. Importación de librerías necesarias

In [1]:
import os
import fitz  # PyMuPDF
import re
import json
from typing import List
from pydantic import BaseModel
import pandas as pd
from sklearn.metrics import confusion_matrix, f1_score, recall_score
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
import requests

In [2]:
os.environ["TOGETHER_API_KEY"] = "tgp_v1_PHdIPUdwUiD4Br7N3HjeZbxeNJuEdTRpW9LrfkFWZe8"

### 📄 2. Cargar y leer texto desde el PDF

In [3]:
from pathlib import Path

import fitz  # PyMuPDF

ruta_facturas = Path("data/facturas/")

def extraer_texto_pdf(pdf_path):
    with fitz.open(pdf_path) as pdf:
        texto_factura = ""
        for pagina in pdf:
            texto_factura += pagina.get_text()
    return texto_factura

for pdf_file in ruta_facturas.glob("*.pdf"):
    texto_factura = extraer_texto_pdf(pdf_file)
    print(f"\n📄 Archivo: {pdf_file.name}\n{'-'*40}")
    print(texto_factura[:500])
    print("-"*40)


📄 Archivo: Endesa Factura 06022025.pdf
----------------------------------------
VX10O020-D-11/02/25 N0016126LNNNN
LARS TANKMAR TANKMAR
AV ONZE DE SETEMBRE 1 3 ATC 2
17255 BEGUR
GERONA
Endesa Energía, S.A. Unipersonal. Inscrita en el Registro Mercantil de Madrid. Tomo 12.797, Libro 0, Folio 208,
 Sección 8ª, Hoja M-205.381, CIF A81948077. Domicilio Social: C/Ribera del Loira, nº60 28042 - Madrid.
Endesa Energía, S.A. Unipersonal.
CIF A81948077.
C/Ribera del Loira, nº 60 28042 - Madrid.
DATOS DE LA FACTURA
Nº factura: P25CON005526043
Referencia: 508332855402
Fecha emisión fac
----------------------------------------


### 🧾 3. Definir el esquema de los datos con Pydantic

In [4]:
from pydantic import BaseModel
from datetime import datetime
from typing import Optional

class FacturaElectrica(BaseModel):
    numero_factura: Optional[str]
    fecha_emision: Optional[datetime]
    periodo_inicio: Optional[datetime]
    periodo_fin: Optional[datetime]
    consumo_total_kwh: Optional[float]
    potencia_punta_kw: Optional[float]
    potencia_valle_kw: Optional[float]
    importe_total: Optional[float]


In [5]:
esquema_json = FacturaElectrica.schema_json(indent=2)
print(esquema_json)

{
  "properties": {
    "numero_factura": {
      "anyOf": [
        {
          "type": "string"
        },
        {
          "type": "null"
        }
      ],
      "title": "Numero Factura"
    },
    "fecha_emision": {
      "anyOf": [
        {
          "format": "date-time",
          "type": "string"
        },
        {
          "type": "null"
        }
      ],
      "title": "Fecha Emision"
    },
    "periodo_inicio": {
      "anyOf": [
        {
          "format": "date-time",
          "type": "string"
        },
        {
          "type": "null"
        }
      ],
      "title": "Periodo Inicio"
    },
    "periodo_fin": {
      "anyOf": [
        {
          "format": "date-time",
          "type": "string"
        },
        {
          "type": "null"
        }
      ],
      "title": "Periodo Fin"
    },
    "consumo_total_kwh": {
      "anyOf": [
        {
          "type": "number"
        },
        {
          "type": "null"
        }
      ],
      "title": 

<ipython-input-5-21c62e08e257>:1: PydanticDeprecatedSince20: The `schema_json` method is deprecated; use `model_json_schema` and json.dumps instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  esquema_json = FacturaElectrica.schema_json(indent=2)


### 🧩 5. Fragmentación semántica y generación de embeddings

In [6]:
import re
from typing import List
from sentence_transformers import SentenceTransformer

# 📌 Cargar modelo de embeddings
modelo_embeddings = SentenceTransformer("all-MiniLM-L6-v2")

# Nueva función para fragmentar texto por secciones específicas
def chunk_por_secciones(texto: str, secciones: List[str]) -> List[str]:
    # Crear patrón regex que detecte los títulos de secciones
    patron = '|'.join([re.escape(sec) for sec in secciones])
    indices = [m.start() for m in re.finditer(patron, texto)]
    indices.append(len(texto))  # Para el último chunk

    chunks = []
    for i in range(len(indices)-1):
        chunk = texto[indices[i]:indices[i+1]].strip()
        if chunk:
            chunks.append(chunk)
    return chunks

# Lista de secciones típicas en factura eléctrica
secciones = [
    "Datos de la factura",
    "Número de factura",
    "Fecha de emisión",
    "Periodo de facturación",
    "Consumo eléctrico",
    "Consumo total",
    "Potencia contratada",
    "Total"
]

# Fragmentar el texto de la factura por secciones
fragmentos = chunk_por_secciones(texto_factura, secciones)

# 📌 Generar embeddings a partir de los fragmentos obtenidos
embeddings = modelo_embeddings.encode(fragmentos, convert_to_tensor=False)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


### 🧩 5. Guardar embeddings en ChromaDB

In [8]:
import chromadb

# Inicializar el cliente de ChromaDB (almacenamiento local por defecto)
client = chromadb.Client()

# Crear o obtener una colección
coleccion = client.get_or_create_collection("facturas_electricas")

# Guardar los embeddings y fragmentos en la colección
ids = [f"fragmento_{i}" for i in range(len(fragmentos))]
coleccion.add(
    embeddings=embeddings.tolist(),
    documents=fragmentos,
    ids=ids
)

print(f"Se han guardado {len(fragmentos)} fragmentos en ChromaDB.")

Se han guardado 3 fragmentos en ChromaDB.


In [9]:
def buscar_contexto_chroma(pregunta: str, modelo_embedding, collection, top_k=3) -> str:
    embedding = modelo_embedding.encode(pregunta).tolist()

    resultados = collection.query(query_embeddings=[embedding], n_results=top_k)

    if not resultados['documents'] or not resultados['documents'][0]:
        return None

    return "\n".join(resultados['documents'][0])

def responder_pregunta_con_together_chroma(pregunta):
    contexto = buscar_contexto_chroma(pregunta, modelo_embeddings, coleccion)

    if not contexto:
        return "No se encontró información relevante en los documentos cargados."

    prompt = f"""Contesta la siguiente pregunta usando solo la información proporcionada en el contexto.
Si no está en el contexto, responde que no tienes suficiente información.

Contexto:
{contexto}

Pregunta: {pregunta}
Respuesta:"""

    response = requests.post(
        "https://api.together.xyz/v1/chat/completions",
        headers={
            "Authorization": f"Bearer {os.environ['TOGETHER_API_KEY']}",
            "Content-Type": "application/json"
        },
        json={
            "model": "mistralai/Mixtral-8x7B-Instruct-v0.1",
            "messages": [{"role": "user", "content": prompt}],
            "temperature": 0.3,
            "max_tokens": 512
        }
    )

    if response.status_code == 200:
        resultado = response.json()
        return resultado["choices"][0]["message"]["content"]
    else:
        return f"Error al generar respuesta: {response.status_code} - {response.text}"


In [10]:
respuesta = responder_pregunta_con_together_chroma("¿Cuál es el importe total en la factura?")
print(respuesta)


 El importe total en la factura es de 66,321 kWh.

Explicación:
La pregunta pregunta por el "importe total en la factura", pero el contexto proporcionado no contiene ninguna información sobre el importe total de la factura en términos monetarios. Sin embargo, sí proporciona información sobre el consumo de energía en kilovatios-hora (kWh). Por lo tanto, la respuesta a la pregunta debe ser el valor de consumo de energía proporcionado en el contexto, que es de 66,321 kWh.


In [11]:
respuesta = responder_pregunta_con_together_chroma("¿Cuál es el periodo de facturación?")
print(respuesta)

 El periodo de facturación es del 31/12/2024 al 31/01/2025 (31 días).


In [12]:
respuesta = responder_pregunta_con_together_chroma("¿Cual es el consumo eléctrico?")
print(respuesta)

 El consumo eléctrico es de 66,321 kWh según la información proporcionada en el contexto. Esto se calcula utilizando el consumo horario real proporcionado por la distribuidora eléctrica.


In [13]:
respuesta = responder_pregunta_con_together_chroma("¿Cuál es el número de factura?")
print(respuesta)

 El contexto no proporciona la información solicitada directamente. No tengo suficiente información para responder a esta pregunta sin inferir o adivinar. La información proporcionada se centra en los detalles de consumo, datos del contrato y destino del importe de la factura, pero no incluye el número de factura.


In [14]:
from pydantic import BaseModel
from typing import Optional
from datetime import datetime
import json
import re

# 2. Función para extraer el primer bloque JSON de una cadena de texto
def extraer_bloque_json(texto):
    match = re.search(r"\{.*?\}", texto, re.DOTALL)
    if match:
        return match.group()
    return None

# 3. Prompt para solicitar los datos estructurados al LLM
prompt_json = """
Extrae los siguientes datos de la factura eléctrica y devuélvelos en formato JSON válido.

Devuelve el resultado en este formato JSON:

{
  "n factura": "string",
  "fecha_emision": "YYYY-MM-DDTHH:MM:SS",
  "periodo_inicio": "YYYY-MM-DDTHH:MM:SS",
  "periodo_fin": "YYYY-MM-DDTHH:MM:SS",
  "consumo_total_kwh": float,
  "potencia_punta_kw": float,
  "potencia_valle_kw": float,
  "importe_total": float
}

Usa solo los datos que estén claramente presentes. Si no aparece algún dato, indica "No tengo suficiente información".

Devuelve exclusivamente el JSON, sin explicaciones.
"""

# 4. Obtener respuesta del sistema RAG
respuesta_json_llm = responder_pregunta_con_together_chroma(prompt_json)

# 5. Extraer y limpiar JSON antes de validar
bloque_json = extraer_bloque_json(respuesta_json_llm)

if bloque_json:
    try:
        datos = json.loads(bloque_json)

        # Limpiar valores con "No tengo suficiente información"
        for k, v in datos.items():
            if isinstance(v, str) and "no tengo" in v.lower():
                datos[k] = None

        # 6. Validar con Pydantic
        factura = FacturaElectrica(**datos)
        print("✅ JSON válido y compatible con FacturaElectrica:")
        print(factura.model_dump_json(indent=2))

    except Exception as e:
        print("❌ Error al validar con Pydantic:", e)
        print("Contenido JSON:", bloque_json)
else:
    print("❌ No se encontró un bloque JSON en la respuesta.")


❌ Error al validar con Pydantic: 1 validation error for FacturaElectrica
numero_factura
  Field required [type=missing, input_value={'n factura': None, 'fech... 'importe_total': 50.65}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
Contenido JSON: {
  "n factura": "No tengo suficiente información",
  "fecha_emision": "2025-02-13T00:00:00",
  "periodo_inicio": "2024-12-31T00:00:00",
  "periodo_fin": "2025-01-31T00:00:00",
  "consumo_total_kwh": 66.321,
  "potencia_punta_kw": 6.928,
  "potencia_valle_kw": 6.928,
  "importe_total": 50.65
}


In [15]:
# prompt: GENERAME UNA BATERIA DE PREGUNTAS SOBRE CUAL ES CADA UNO DE ESTOS PUNTOS
#   "numero_factura": "string",
#   "fecha_emision": "YYYY-MM-DDTHH:MM:SS",
#   "periodo_inicio": "YYYY-MM-DDTHH:MM:SS",
#   "periodo_fin": "YYYY-MM-DDTHH:MM:SS",
#   "consumo_total_kwh": float,
#   "potencia_punta_kw": float,
#   "potencia_valle_kw": float,
#   "importe_total": float
# Y COMPARALO CON EL RESULTADO DEL JSON respuesta_json_llm

esquema_pydantic = """
{
  "numero_factura": "string",
  "fecha_emision": "YYYY-MM-DDTHH:MM:SS",
  "periodo_inicio": "YYYY-MM-DDTHH:MM:SS",
  "periodo_fin": "YYYY-MM-DDTHH:MM:SS",
  "consumo_total_kwh": "float",
  "potencia_punta_kw": "float",
  "potencia_valle_kw": "float",
  "importe_total": "float"
}
"""

def generar_preguntas_y_comparar(esquema_pydantic, respuesta_json_llm):
    """
    Genera preguntas sobre cada campo del esquema Pydantic
    y compara las respuestas del LLM con los valores extraídos.
    """
    esquema_dict = json.loads(esquema_pydantic)
    try:
        respuesta_dict_llm = json.loads(respuesta_json_llm)
        print("\n--- Comparación de Valores Extraídos ---")
        for campo, tipo in esquema_dict.items():
            pregunta = f"¿Cuál es el valor del campo '{campo}'?"
            print(f"\nPregunta: {pregunta}")


            valor_llm = respuesta_dict_llm.get(campo, "N/A")
            print(f"Valor extraído por el LLM (del JSON final): {valor_llm}")

    except json.JSONDecodeError:
        print("Error: La respuesta del LLM no es un JSON válido.")
    except Exception as e:
        print(f"Ocurrió un error: {e}")

# Llama a la función con tu esquema y la respuesta JSON del LLM
generar_preguntas_y_comparar(esquema_pydantic, respuesta_json_llm)




--- Comparación de Valores Extraídos ---

Pregunta: ¿Cuál es el valor del campo 'numero_factura'?
Valor extraído por el LLM (del JSON final): N/A

Pregunta: ¿Cuál es el valor del campo 'fecha_emision'?
Valor extraído por el LLM (del JSON final): 2025-02-13T00:00:00

Pregunta: ¿Cuál es el valor del campo 'periodo_inicio'?
Valor extraído por el LLM (del JSON final): 2024-12-31T00:00:00

Pregunta: ¿Cuál es el valor del campo 'periodo_fin'?
Valor extraído por el LLM (del JSON final): 2025-01-31T00:00:00

Pregunta: ¿Cuál es el valor del campo 'consumo_total_kwh'?
Valor extraído por el LLM (del JSON final): 66.321

Pregunta: ¿Cuál es el valor del campo 'potencia_punta_kw'?
Valor extraído por el LLM (del JSON final): 6.928

Pregunta: ¿Cuál es el valor del campo 'potencia_valle_kw'?
Valor extraído por el LLM (del JSON final): 6.928

Pregunta: ¿Cuál es el valor del campo 'importe_total'?
Valor extraído por el LLM (del JSON final): 50.65
